## Dashboard for displaying visualizations

In [84]:
import numpy as np
import pandas as pd
import altair as alt
from dash import dash, dcc, html, Input, Output
from altair import datum
import os
import json

In [34]:
nav_events = pd.read_csv('hackathon_alternate_data/navigation_events.csv')
disc = pd.read_csv('hackathon_alternate_data/additional/discussions.csv')
disc_dates = pd.read_csv('hackathon_alternate_data/additional/discussion_topics.csv')

In [38]:
disc['timestamp'] = pd.to_datetime(disc['timestamp'])
disc['date'] = disc['timestamp'].dt.normalize()#date
disc_merged = disc.merge(disc_dates[['title','todo_date_date']],how='inner',left_on='discussion_topic_title',right_on='title')

In [121]:
# Plot hist of assignment submission dates
def plot_altair(discussion_topic):
    chart = alt.Chart(disc_merged).mark_bar().encode(
        alt.X('date:T').title("Date"),
        alt.Y('count(x):N').title("Number of Submissions")).transform_filter(datum.discussion_topic_title == discussion_topic)
    line = chart.mark_rule(color='red').encode(x = 'todo_date_date:T')
    return (chart+line).to_html()

In [126]:
# Simon's cumulative assignment view viz + data cleaning
assignments = pd.read_csv("hackathon_alternate_data/additional/assignments.csv")
navigation_events = pd.read_csv("hackathon_alternate_data/navigation_events.csv")
enrollments = pd.read_csv("hackathon_alternate_data/additional/enrollments.csv")

# drop strange/na columns
navigation_events.drop(columns=["ed_app", "type", "action", "course_offering_id", "statement_type", "statement_version", "event__object_type", "event__object_extensions_asset_subtype", "event__object_extensions_entity_id", "event__referrer", "event__extensions_request_url", "event__attachment_type"], inplace=True)
# drop learners that are not enrolled in the course
navigation_events = navigation_events.iloc[np.in1d(navigation_events.actor_id, enrollments.user_id), :]
navigation_events = navigation_events.query("actor_id != 'LEARNER_48'")
navigation_events.loc[:, "event_time"] = pd.to_datetime(navigation_events.loc[:, "event_time"], format='ISO8601').dt.normalize()

df_assignments = assignments.drop(columns=["unlock_at", "lock_at", "grading_type", "position"]).query("has_submitted_submissions == True")
df_assignments["due_at"] = pd.to_datetime(df_assignments["due_at"])

def plot_cumulative_assignment_view(assignment_name):
    
    ## Drawing
    tmp_assignment = df_assignments.query("name == @assignment_name")
    tmp_assignment_due_date = tmp_assignment.due_at
    tmp_first_accessed = navigation_events.query("object_id == @tmp_assignment.id.iloc[0]")[["actor_id", "event_time"]]
    tmp_first_accessed = tmp_first_accessed.sort_values(["actor_id", "event_time"]).groupby("actor_id").first().sort_values("event_time")
    
    tmp_first_accessed = tmp_first_accessed.reset_index()
    tmp_first_accessed['event_time'] = pd.to_datetime(tmp_first_accessed['event_time'])
    chart = alt.Chart(tmp_first_accessed).transform_window(
        ecdf="cume_dist()",
        sort=[{"field": "event_time"}],
    ).mark_line().encode( #interpolate="step-after"
        x=alt.X("event_time:T").scale(domain=[tmp_first_accessed.event_time.iloc[0] - pd.Timedelta(days=7),  tmp_assignment.due_at.iloc[0] + pd.Timedelta(days=7)]),
        y=alt.Y("ecdf:Q").scale(domain=[0, 1.1]).title("Assignment View %"), 
    )
    chart = chart + alt.Chart(tmp_assignment).mark_rule().encode(x = alt.X("due_at").title(None))
    return chart.properties(width=400, height=200).to_html()

#plot_cumulative_assignment_view(Assignment 1")

In [ ]:
# Set up dashboard
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

assignments = ['Assignment 1', 'Assignment 2', 'Assignment 3 (option A)', 
               'Assignment 3 (option B)','Assignment 3 (option C)','Assignment 3 (option D)']
discussion_topics = disc['discussion_topic_title'].unique()

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Home', children=[html.H1('Course-123 Instructor Analytics'),
            html.P('Welcome to your instructor dashboard! Navigate through the tabs for insights on student engagement and performance'),
            dcc.Graph(
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [2, 4, 3],
                            'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [5, 4, 3],
                         'type': 'bar', 'name': 'Montréal'},
                    ]
                }
            )]),
        
        dcc.Tab(label='Discussion Posts', children=[
            dcc.Dropdown(
            id='discussion', value='Discussion 1: The meaning of eLearning',
            options=[{'label': i, 'value': i} for i in discussion_topics]),
        html.Iframe(
            id='disc-scatter',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_altair('Discussion 1: The meaning of eLearning'))]),
        
        dcc.Tab(label='Assignments', children=[
            dcc.Dropdown(
            id='assignment_name', value='Assignment 1',
            options=[{'label': i, 'value': i} for i in assignments]),
        html.Iframe(
            id='a-scatter',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_cumulative_assignment_view('Assignment 1'))
        ]),
        
        dcc.Tab(label='Individual Students', children=[
            dcc.Graph(
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [2, 4, 3],
                            'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [5, 4, 3],
                         'type': 'bar', 'name': 'Montréal'},
                    ]
                }
            )
        ]),
    ])    
])

# Callback for updating the 'Discussion'
@app.callback(
    Output('disc-scatter', 'srcDoc'),
    Input('discussion', 'value')
)
def update_disc_scatter(discussion_topic):
    return plot_altair(discussion_topic)

# Callback for updating the 'Assignments'
@app.callback(
    Output('a-scatter', 'srcDoc'),
    Input('assignment_name', 'value')
)
def update_a_scatter(assignment_name):
    return plot_cumulative_assignment_view(assignment_name)

if __name__ == '__main__':
    app.run_server(debug=True)